In [1]:
"""
In this script, we will
"""

import torch 
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
from sort import sort
from time import time
from collections import defaultdict
import threading
# import required libraries
from vidgear.gears import CamGear
from vidgear.gears import WriteGear
import cv2
"""
Load the model with a certain confidence threshold (TBD)
"""
model = torch.hub.load('.','custom','best.pt',source='local')
model.conf = .4 #Minimum .5 confidence threshold

/home/erick/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
YOLOv5 🚀 b62ae3263 Python-3.8.10 torch-1.13.0+cu117 CPU

Fusing layers... 
Model summary: 212 layers, 20893344 parameters, 0 gradients
Adding AutoShape... 


In [9]:
"""
Initializing values for tracking and video capture:
"""
#Store tracking info per video
tracker=sort.Sort() 
ids = defaultdict(set)
#.ttf file, font-size
myFont = ImageFont.truetype('ostrich-regular.ttf', 15)
# define and open video source
stream = CamGear(source="/dev/video0", logging=True).start()

# [WARNING] Change your YouTube-Live Stream Key here:
YOUTUBE_STREAM_KEY = "1szm-jm2t-m45s-ygw9-1rbx"

22:21:33 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!


In [10]:
# define required FFmpeg parameters for your writer
output_params = {
    "-clones": ["-f", "lavfi", "-i", "anullsrc"],
    "-vcodec": "libx264",
    "-preset": "medium",
    "-b:v": "4500k",
    "-bufsize": "512k",
    "-pix_fmt": "yuv420p",
    "-f": "flv",
}
# Define writer with defined parameters
writer = WriteGear(
    output="rtmp://a.rtmp.youtube.com/live2/{}".format(YOUTUBE_STREAM_KEY),
    logging=True,
    **output_params
)
lock = threading.Lock()
def sendFrameToStream(frame, model_thread):
    while(model_thread.is_alive()):
        lock.acquire()
        writer.write(frame)
        lock.release()

22:21:34 ::    Helper     :: WARNING  :: Specified directory `/home/erick/Documents/college/class/EE498/traffic-detection-yolov5/rtmp:/a.rtmp.youtube.com/live2` doesn't exists or valid.
22:21:34 ::   WriteGear   ::   INFO   :: `rtmp://a.rtmp.youtube.com/live2/1szm-jm2t-m45s-ygw9-1rbx` isn't a valid system path or directory. Skipped!
22:21:34 ::   WriteGear   ::  DEBUG   :: Output Parameters: `{'-clones': ['-f', 'lavfi', '-i', 'anullsrc'], '-vcodec': 'libx264', '-preset': 'medium', '-b:v': '4500k', '-bufsize': '512k', '-pix_fmt': 'yuv420p', '-f': 'flv'}`
22:21:34 ::   WriteGear   ::  DEBUG   :: Compression Mode is enabled therefore checking for valid FFmpeg executable.
22:21:34 ::    Helper     ::  DEBUG   :: Final FFmpeg Path: ffmpeg
22:21:34 ::    Helper     ::  DEBUG   :: FFmpeg validity Test Passed!
22:21:34 ::    Helper     ::  DEBUG   :: Found valid FFmpeg Version: `b'4.2.7-0ubuntu0.1'` installed on this system
22:21:34 ::   WriteGear   ::  DEBUG   :: Found valid FFmpeg executable

In [11]:
def model_image(frame):
    """
    Takes an image frame and returns the traffic count in the frame 
    """
    results = model(frame)
    #Retrieving boundingbox data as dataframe 
    #Dataframe Format (xyxy attribute):
    #(xmin,ymin,xmax,ymax,confidence,label_value,label)
    data = results.pandas().xyxy[0]        
    trafficType=data["name"]
    data = data.iloc[:][:5].to_numpy()
    data = data[:,:5].astype('float64')

    #Updated with ids on the camera
    track_res = tracker.update(data)

    if(track_res.size!=0):
        for index,detection in enumerate(track_res[::-1]):
            #If the detection is new, add the ID to the
            #appropriate vehicle/pedestrian set
            #COUNT=LENGTH
            if(detection[-1] not in ids[trafficType[index]]):
                ids[trafficType[index]].add(detection[-1])

    #Image WITH BOX PREDICTIONS AND COUNT
    im = Image.fromarray(results.render(labels=False)[0])

    #Draw the count on the video frame
    im_draw = ImageDraw.Draw(im)
    draw_text="Traffic Counts\n"
    for trafficType, id_set in ids.items():
        draw_text+=trafficType+"'s counted=" +str(len(id_set))+"\n"

    im_draw.multiline_text((0,
                      0), 
                 draw_text,
                 fill='white',font=myFont,
                 anchor = None, spacing = 0,
                 align="left",direction=None,
                 features=None,language=None,
                 stroke_width=1, stroke_fill="black")
    lock.acquire()
    frame = im
    lock.release()

In [15]:
# loop over
while True:

    # read frames from stream
    frame = stream.read()
    # check for frame if Nonetype
    if frame is None:
        break
    
    
    thread1 = threading.Thread(target=model_image, kwargs={'frame':frame})
    thread1.start()
    thread2 = threading.Thread(target=sendFrameToStream, kwargs={'frame':frame, 'model_thread':thread1})
    thread2.start()
    
    
    thread1.join()
    thread2.join()


22:24:10 ::   WriteGear   ::  DEBUG   :: InputFrame => Height:480 Width:640 Channels:3 Datatype:uint8
22:24:10 ::   WriteGear   ::  DEBUG   :: Executing FFmpeg command: `ffmpeg -y -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt bgr24 -i - -f lavfi -i anullsrc -vcodec libx264 -preset medium -b:v 4500k -bufsize 512k -pix_fmt yuv420p -f flv -crf 18 rtmp://a.rtmp.youtube.com/live2/1szm-jm2t-m45s-ygw9-1rbx`
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enabl

KeyboardInterrupt: 

In [16]:
# safely close video stream
stream.stop()

# safely close writer
writer.close()

22:35:51 ::    CamGear    ::  DEBUG   :: Terminating processes.
22:35:51 ::   WriteGear   ::  DEBUG   :: Terminating WriteGear Processes.


In [23]:
from __future__ import print_function

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


def create(title):
    """
    Creates the Sheet the user has access to.
    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
        """
    creds, _ = google.auth.default()
    # pylint: disable=maybe-no-member
    try:
        service = build('sheets', 'v4', credentials=creds)
        spreadsheet = {
            'properties': {
                'title': title
            }
        }
        spreadsheet = service.spreadsheets().create(body=spreadsheet,
                                                    fields='spreadsheetId') \
            .execute()
        print(f"Spreadsheet ID: {(spreadsheet.get('spreadsheetId'))}")
        return spreadsheet.get('spreadsheetId')
    except HttpError as error:
        print(f"An error occurred: {error}")
        return error


create("smartTrafficCameraCarData")

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [ ]:
from google.cloud import storage


def authenticate_implicit_with_adc(project_id="your-google-cloud-project-id"):
    """
    When interacting with Google Cloud Client libraries, the library can auto-detect the
    credentials to use.

    // TODO(Developer):
    //  1. Before running this sample,
    //  set up ADC as described in https://cloud.google.com/docs/authentication/external/set-up-adc
    //  2. Replace the project variable.
    //  3. Make sure that the user account or service account that you are using
    //  has the required permissions. For this sample, you must have "storage.buckets.list".
    Args:
        project_id: The project id of your Google Cloud project.
    """

    # This snippet demonstrates how to list buckets.
    # *NOTE*: Replace the client created below with the client required for your application.
    # Note that the credentials are not specified when constructing the client.
    # Hence, the client library will look for credentials using ADC.
    storage_client = storage.Client(project=project_id)
    buckets = storage_client.list_buckets()
    print("Buckets:")
    for bucket in buckets:
        print(bucket.name)
    print("Listed all storage buckets.")
